## 長いアブストラクトの処理


In [ ]:
import pandas as pd
import os
import re
from tqdm.auto import tqdm


# --- 1. 設定 ---
INPUT_FILE = "data/processed/training_dataset_abstract.csv"
OUTPUT_FILE = "data/processed/training_dataset_abstract_cleaned.csv"

# アブストラクトの終わり（＝本文の始まり）を示す可能性が非常に高いキーワード
# （大文字と小文字を区別しないように、すべて小文字で定義）
STOP_WORDS = [
    'introduction',
    'keywords',
    'key words',
    'references',
    'acknowledgments',
    'acknowledgements',
    'bibliography',
    'pubmed abstract',
    'publisher full text',
    'full text'
]

# 正規表現パターンを作成 ( | で区切る)
# \b は単語の境界を意味し、"introduction" が "subintroduction" にマッチしないようにする
STOP_PATTERN = re.compile(r'\b(' + '|'.join(STOP_WORDS) + r')\b', re.IGNORECASE)

print("Settings and stop words defined.")

In [ ]:
# --- 2. クリーニング関数の定義 ---

def clean_abstract(text):
    """
    テキストを受け取り、ストップワードが最初に出現した位置で
    テキストを切り捨てる（クリーニングする）
    """
    if not isinstance(text, str):
        return text
    
    # ストップワードを検索
    match = STOP_PATTERN.search(text)
    
    if match:
        # ストップワードが見つかった場合
        # その単語が始まる位置（match.start()）までのテキストを返す
        return text[:match.start()]
    else:
        # ストップワードが見つからなかった場合
        # テキストは汚染されていないと判断し、そのまま返す
        return text

# テスト実行
test_text = "This is abstract. \n\n Keywords: NLP, ML. \n\n This is body..."
print(f"Original: {test_text}")
print(f"Cleaned:  {clean_abstract(test_text)}")

In [ ]:
# --- 3. データセットの読み込みと前処理の実行 ---
print(f"Loading dataset from {INPUT_FILE}...")
df = pd.read_csv(INPUT_FILE)
df = df.dropna(subset=['abstract_a', 'abstract_b'])

print("Applying cleaning function to 'abstract_a'...")
df['abstract_a_cleaned'] = df['abstract_a'].progress_apply(clean_abstract)

print("Applying cleaning function to 'abstract_b'...")
df['abstract_b_cleaned'] = df['abstract_b'].progress_apply(clean_abstract)

print("Cleaning complete.")

In [ ]:
# --- 4. 最終データセットの保存 ---

# 元のアブストラクトを削除し、クリーニング後のものだけを残す
df_final = df.drop(columns=['abstract_a', 'abstract_b'])
df_final = df_final.rename(columns={
    'abstract_a_cleaned': 'abstract_a',
    'abstract_b_cleaned': 'abstract_b'
})

# 必要なカラムの順序に並び替え
df_final = df_final[['abstract_a', 'abstract_b', 'label', 'data_paper_doi']]

print(f"Saving cleaned dataset to {OUTPUT_FILE}...")
df_final.to_csv(OUTPUT_FILE, index=False)
print("Save complete.")